In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('trybik.jpg')
img_pattern = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, bin_img = cv2.threshold(img_pattern, 240, 255, cv2.THRESH_BINARY)

bin_img = cv2.bitwise_not(bin_img)

bin_img = cv2.medianBlur(bin_img, 3)

plt.imshow(img)
plt.title("Original Image")
plt.axis('off')
plt.show()

plt.imshow(img_pattern, cmap='gray')
plt.title("Grayscale Image")
plt.axis('off')
plt.show()

plt.imshow(bin_img, cmap='gray')
plt.title("Binary image")
plt.axis('off')
plt.show()

contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
img_contours = cv2.drawContours(img, contours, -1, (255, 0, 0)),
plt.imshow(img_contours[0])
plt.title('Contours')
plt.axis('off')
plt.show()

In [ ]:
sobelx = cv2.Sobel(img_pattern, cv2.CV_64F, 1, 0, ksize=5)
sobely = cv2.Sobel(img_pattern, cv2.CV_64F, 0, 1, ksize=5)

amplitude = np.sqrt(sobelx**2 + sobely**2)
orientation = np.arctan2(sobely, sobelx) * (180 / np.pi) % 360

orientation = (np.degrees(np.arctan2(sobely, sobelx)) + 360).astype(int) % 360
moments = cv2.moments(img_pattern, True)
m00, m01, m10 = moments['m00'], moments['m01'], moments['m10']
ref_point = (int(m10/m00), int(m01/m00))

contours_merged = []
Rtable = [[] for _ in range(360)]

for c in contours:
    contours_merged.extend(c.squeeze())
for i, j in contours_merged:
    r = np.sqrt((i - ref_point[0])**2 + (j - ref_point[1])**2)
    alpha = np.arctan2(j - ref_point[1], i - ref_point[0])
    if i < orientation.shape[0] and j < orientation.shape[1]:
        Rtable[int(orientation[i,j])].append((r, alpha))

non_empty = sum(1 for r in Rtable if len(r) > 0)
print(f"Niepustych kątów w Rtable: {non_empty}")


In [ ]:
def conv_image(image_path):
    img_rgb = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return img_rgb, img_gray

def gradients(img_gray):
    sobelx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=5)
    return sobelx, sobely

def orientation_and_amplitude(sobelx, sobely):
    orientation = (np.degrees(np.arctan2(sobely, sobelx)) + 360).astype(int) % 360
    amplitude = np.sqrt(sobelx**2 + sobely**2)
    amplitude /= np.max(amplitude)
    return orientation, amplitude

def hough_transform(orientation, amplitude, Rtable, shape):
    hough_space = np.zeros(shape, dtype=np.int32)
    for i in range(orientation.shape[0]):
        for j in range(orientation.shape[1]):
            if amplitude[i, j] > 0.1: 
                phi = orientation[i, j]
                if 0 <= int(phi) < 360 and Rtable[int(phi)]: 
                    for r, alpha in Rtable[int(phi)]:
                        x, y = int(i - r * np.cos(alpha)), int(j - r * np.sin(alpha))
                        if 0 <= x < shape[0] and 0 <= y < shape[1]:
                            hough_space[x, y] += 1
    
    print(f"Max value in Hough space: {np.max(hough_space)}")
    print(f"Number of non-zero entries: {np.count_nonzero(hough_space)}")
    return hough_space

def plot(image, hough, contours, ref_point):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    peaks = zero_maximum(hough.copy(), 115, 5)
    for peak in peaks:
        x_offsets = [p[0] + peak[1] - ref_point[0] for p in contours]
        y_offsets = [p[1] + peak[0] - ref_point[1] for p in contours]
        plt.plot(x_offsets, y_offsets, 'ro', markersize=0.4)
    plt.title('Detected patterns')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    if np.max(hough) > 0:
        log_hough = np.log1p(hough.astype(float))  
        plt.imshow(log_hough, cmap='gray')

    plt.tight_layout()
    plt.show()


def zero_maximum(hough_space, delta=30, num_peaks=5):
    peaks = []
    for _ in range(num_peaks):
        m = np.unravel_index(np.argmax(hough_space), hough_space.shape)
        if hough_space[m] == 0:
            break
        peaks.append(m)
        x, y = m
        hough_space[max(x-delta, 0):min(x+delta, hough_space.shape[0]),
                    max(y-delta, 0):min(y+delta, hough_space.shape[1])] = 0
    return peaks



In [ ]:
img_rgb, img_gray = conv_image('trybiki2.jpg')
sobelx, sobely = gradients(img_gray)
orientation, amplitude = orientation_and_amplitude(sobelx, sobely)

hough_space = hough_transform(orientation, amplitude, Rtable, img_gray.shape)
plot(img_rgb, hough_space, contours_merged, ref_point)